# Projeto: UNB - AutoPark

### O dataset



A preparação dos dados sera feita para que tenhamos todas as informaçoes relevantes de cada uma das imagens em um unico registro. Os metadados de cada uma das imagens estarão reunidados em um unico Dataframe e sera pesistido através de um arquivo CSV.

As imagens do dataset serão processadas para que seu lado de maior dimensão seja receba um redimensionamento para a maior dimensão entre todas do conjunto de dados. Para a dimensão da imagem que não esta na maior possivel do dataset será aplicado um padding com a cor preta. Esta nova imagem será salva em um novo diretório ja pré processada.


Assim cada um dos registros sera feito desta forma:


relative_path   | parkinglot_name | date | weather | status 
--------- | ------ | ------ | ------ | ------ 
str | str | str | str | bool


In [ ]:
import os
import ipyplot
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import imutils
DATA_PATH = "/home/delll/fga/veiculos_autonomos/datasets/PKLot/PKLotSegmented"
NEW_DATA_PATH = "/home/delll/fga/veiculos_autonomos/datasets/PKLot/ready2go/"

# Maiores dimensões possiveis na base de dados
bigger_w = 176
bigger_h = 93

In [ ]:
def pad_image(img, bigger_w, bigger_h):
    
    padding_sides = (bigger_w - img.shape[0])/ 2
    if(padding_sides.is_integer() ==  False):
        padding_sides = int(padding_sides - 0.5)
    else:
        padding_sides = int(padding_sides)

    padding_top = (bigger_h - img.shape[1])/2    
    if(padding_top.is_integer() ==  False):
        padding_top = int(padding_top - 0.5)
    else:
        padding_top = int(padding_top)

    result = np.ones([bigger_w, bigger_h, 3])
    try:
        result[padding_sides: padding_sides+img.shape[0], padding_top: padding_top + img.shape[1]] = np.array(img)
    except:
        return []
    return result




def read_dataset_dir(parkinglot_name, weather, bigger_w, bigger_h, new_images_path):
    images_path = DATA_PATH + "/"+ parkinglot_name + "/" + weather + "/"

    for date in os.listdir(images_path):
        dir_info_list= []

        for state in os.listdir(images_path + date +"/"):

            for file in os.listdir(images_path + date +"/" + state):
                # path to image file
                
                file_path = images_path + date +"/" + state + "/" + file
                new_file_path = os.path.join(new_images_path , file)

                img = cv2.imread(file_path)
                # make padding and save image
                pw, ph = img.shape[0]/176, img.shape[1]/93
                n_max_size = (110, 58)
                if (pw > ph):
                    img = imutils.resize(img, height=110)
                elif (ph >= pw):
                    img = imutils.resize(img, width=58)
                new_img = pad_image(img, n_max_size[0], n_max_size[1])
                if new_img == []:
                    continue  
                cv2.imwrite(new_file_path, new_img)
                dir_info_list.append([ new_file_path,  parkinglot_name, date, weather,  state ])
                
                # should be changed to a log function
                print("Leitura da imagem com sucesso: ", new_file_path.split("/")[-1],  parkinglot_name, date, weather, state)
    return dir_info_list

In [ ]:
puc_cloudy = read_dataset_dir("PUC", "Cloudy", bigger_w, bigger_h, NEW_DATA_PATH)
puc_sunny = read_dataset_dir("PUC", "Sunny", bigger_w, bigger_h, NEW_DATA_PATH)
puc_rainy = read_dataset_dir("PUC", "Rainy", bigger_w, bigger_h, NEW_DATA_PATH)

UFPR04_cloudy = read_dataset_dir("UFPR04", "Cloudy", bigger_w, bigger_h, NEW_DATA_PATH)
UFPR04_sunny  = read_dataset_dir("UFPR04", "Sunny", bigger_w, bigger_h, NEW_DATA_PATH)
UFPR04_rainy  = read_dataset_dir("UFPR04", "Rainy", bigger_w, bigger_h, NEW_DATA_PATH)            

UFPR05_cloudy = read_dataset_dir("UFPR05", "Cloudy", bigger_w, bigger_h, NEW_DATA_PATH)
UFPR05_sunny = read_dataset_dir("UFPR05", "Sunny", bigger_w, bigger_h, NEW_DATA_PATH)
UFPR05_rainy = read_dataset_dir("UFPR05", "Rainy", bigger_w, bigger_h, NEW_DATA_PATH)


info_list = puc_sunny + puc_cloudy + puc_rainy + UFPR04_cloudy + UFPR04_sunny + UFPR04_rainy + UFPR05_cloudy + UFPR05_sunny + UFPR05_rainy

In [ ]:
df = pd.DataFrame(data=info_list, columns=[ "relative_path", "parkinglot_name", "date", "weather", "status" ])

df = df.sample(frac=1).reset_index(drop=True)

df.to_csv("data.csv")

In [ ]:
img = cv2.imread(df['relative_path'][6])
plt.imshow(img)
img.shape

# Entendimento dos dados

Após o pré processamento prodemos realizar algumas operações e ferramentas de vizualização para o melhor entendimendo dos dados. Iremos utilizar dos metadados obtidos na etapa anterior deste notebook. Não será necessário o carregamento desta imagens para esta etapa, mas para melhor eficiencia deste notebook iremos carregar as imagens agora. 

In [ ]:
df = pd.read_csv("data.csv")


X = []
y = []

for idx, element in enumerate(df.iterrows()):

    img = cv2.imread(element[1]['relative_path'])

    #normalizando as imagens, colocando em um intervalo de 0 a 1
    X.append(img)
    label = 0 if element[1]['status'] == 'Occupied' else 1  
    y.append(label)

## O Dataframe
Agora, com o data frame carregado podemos ver um exemplo de seus primeiros elementos.

In [ ]:
df.head(10)

# Modelo



In [ ]:
X_train, X_test = X[0 : int(len(X)*0.75)], X[int(len(X)*0.75): len(X)]
y_train, y_test = y[0 : int(len(y)*0.75)], y[int(len(y)*0.75): len(y)]

In [ ]:
X_train, X_test = np.array(X_train), np.array(X_test)
y_train, y_test = np.array(y_train), np.array(y_test)
model = models.Sequential()
model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(110, 58, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=5, 
                    validation_data=(X_test, y_test))

In [ ]:
model.save_weights("model.h5")

In [ ]:
y_pred = model.predict(X_test).ravel()
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
auc = auc(fpr, tpr)

In [ ]:
print("Acuracia obtida:", auc)

In [ ]:
plt.figure(1)

plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='Test Data (area = {:.3f})'.format(auc))
# plt.plot(fpr_train, tpr_train, label='Train Data (area = {:.3f})'.format(auc_train))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()      

In [ ]:
# Teste pratico

In [ ]:
test_image_path = "/home/delll/fga/veiculos_autonomos/datasets/imagens_teste_manual/3.png"

In [ ]:
# img = cv2.imread(test_image_path)
img = cv2.imread(df['relative_path'][6])

pw, ph = img.shape[0]/176, img.shape[1]/93
n_max_size = (110, 58)
if (pw > ph):
    img = imutils.resize(img, height=110)
else:
    img = imutils.resize(img, width=58)
    
padding_sides = (110 - img.shape[0])/ 2
if(padding_sides.is_integer() ==  False):
    padding_sides = int(padding_sides - 0.5)
else:
    padding_sides = int(padding_sides)

padding_top = (58 - img.shape[1])/2    
if(padding_top.is_integer() ==  False):
    padding_top = int(padding_top - 0.5)
else:
    padding_top = int(padding_top)

result = np.ones([110, 58, 3])
result[padding_sides: padding_sides+img.shape[0], padding_top: padding_top + img.shape[1]] = np.array(img)

In [ ]:
img.shape

In [ ]:
result = np.ones([110, 58, 3])
result[padding_sides: padding_sides+img.shape[0], padding_top: padding_top + img.shape[1]] = img
result = result.astype('uint8')

In [ ]:
plt.imshow(result)

In [ ]:
a = result.reshape([1, 110, 58, 3])

In [ ]:
model.predict(a)